<a href="https://colab.research.google.com/github/GudLukas/Lovelace-Team-Project/blob/main/datacleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**To run, execute the code below and then choose the dataset file to be explored.**

In [107]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from zipfile import ZipFile
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
uploaded = files.upload() #manually upload the zip of data??

Saving HeartDataCE101.zip to HeartDataCE101 (8).zip


In [108]:
file_name = "HeartDataCE101.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print("done") #extract the data from the .zip

done


In [109]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')
import plotly.graph_objs as go
import plotly.offline as py
df=pd.read_csv('heart.csv')


In [110]:
df.isnull().sum() #check for null values

,0
age,0
sex,0
cp,0
trestbps,0
chol,0
fbs,0
restecg,0
thalach,0
exang,0
oldpeak,0


In [ ]:
df_dup = df.duplicated().any() #bool checks for any dupe records
print(df_dup)
df = df.drop_duplicates() #rid of duplicates
df_dup = df.duplicated().any()
print(df_dup) #verifies indeed we have no more duplicate values

True
False


In [ ]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
#rename stuff
 #some style, dataframe object = to database.
df.columns = ['Age', 'Sex', 'Chest_pain_type', 'Resting_bp',
              'Cholesterol', 'Fasting_bs', 'Resting_ecg',
              'Max_heart_rate', 'Exercise_induced_angina',
              'ST_depression', 'ST_slope', 'Num_major_vessels',
              'Thallium_test', 'Condition']
df.head()


,Age,Sex,Chest_pain_type,Resting_bp,Cholesterol,Fasting_bs,Resting_ecg,Max_heart_rate,Exercise_induced_angina,ST_depression,ST_slope,Num_major_vessels,Thallium_test,Condition
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [ ]:
#pre-processing
cate_val = [] #categorical columns
cont_val  = [] #numerical/continuous data columns
for column in df.columns:
  if df[column].nunique() <=10:
    cate_val.append(column)
  else:
    cont_val.append(column)
#so the idea here is that if there are less than 10 unique values in a column its a categorical thing, more makes it numerical
#i suppose this might make sense, maybe there are a limited amount of times of things.


In [ ]:
cate_val

['Chest_pain_type',
 'Fasting_bs',
 'Resting_ecg',
 'Exercise_induced_angina',
 'ST_slope',
 'Num_major_vessels',
 'Thallium_test']

In [ ]:
cont_val

['Age', 'Resting_bp', 'Cholesterol', 'Max_heart_rate', 'ST_depression']

In [ ]:
df['Chest_pain_type'].unique() #these are the values chest pain type can hold
#some ML models may think there is an order here in [3,2,1,0], but there isn't in reality

array([3, 2, 1, 0])

In [ ]:
#need to avoid dummy variable trap below
#an example of this is using a tuple to mean Red, Green or Blue. (1,0,0) or a 3-tuple is not good (the trap), as a 2-tuple suffices. (1,0) for Red, (0,1) for Blue and (0,0) for Green.
#apparently scikit is supposed to stop this happening automatically, but i don't know.
cate_val.remove('Sex')
cate_val.remove("Condition") #these are already binary, so dont need
df = pd.get_dummies(df, columns=cate_val, drop_first = True) # so this line is turning each value into a vector, and we drop the first column as a 0 vector can be used to predict one more type of value, as above.



In [ ]:
df.head(5)

,Age,Sex,Resting_bp,Cholesterol,Max_heart_rate,ST_depression,Condition,Chest_pain_type_1,Chest_pain_type_2,Chest_pain_type_3,...,Exercise_induced_angina_1,ST_slope_1,ST_slope_2,Num_major_vessels_1,Num_major_vessels_2,Num_major_vessels_3,Num_major_vessels_4,Thallium_test_1,Thallium_test_2,Thallium_test_3
0,63,1,145,233,150,2.3,1,False,False,True,...,False,False,False,False,False,False,False,True,False,False
1,37,1,130,250,187,3.5,1,False,True,False,...,False,False,False,False,False,False,False,False,True,False
2,41,0,130,204,172,1.4,1,True,False,False,...,False,False,True,False,False,False,False,False,True,False
3,56,1,120,236,178,0.8,1,True,False,False,...,False,False,True,False,False,False,False,False,True,False
4,57,0,120,354,163,0.6,1,False,False,False,...,True,False,True,False,False,False,False,False,True,False


In [ ]:
#feature scaling (also called Z-score normalisation), s.t. each value has a standard dev of 1 and a mean of 0
#apparently, this stops one+ variables being too dominant in the model
#we dont require feature scaling for categorical values because they are all changed to binary values, i think
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
df[cont_val] = st.fit_transform(df[cont_val]) #fit_transform is supposed to do exactly as above wrt to s.d and mean
df.head()

,Age,Sex,Resting_bp,Cholesterol,Max_heart_rate,ST_depression,Condition,Chest_pain_type_1,Chest_pain_type_2,Chest_pain_type_3,...,Exercise_induced_angina_1,ST_slope_1,ST_slope_2,Num_major_vessels_1,Num_major_vessels_2,Num_major_vessels_3,Num_major_vessels_4,Thallium_test_1,Thallium_test_2,Thallium_test_3
0,0.949794,1,0.764066,-0.261285,0.018826,1.084022,1,False,False,True,...,False,False,False,False,False,False,False,True,False,False
1,-1.928548,1,-0.091401,0.067741,1.636979,2.118926,1,False,True,False,...,False,False,False,False,False,False,False,False,True,False
2,-1.485726,0,-0.091401,-0.822564,0.980971,0.307844,1,True,False,False,...,False,False,True,False,False,False,False,False,True,False
3,0.174856,1,-0.661712,-0.203222,1.243374,-0.209608,1,True,False,False,...,False,False,True,False,False,False,False,False,True,False
4,0.285561,0,-0.661712,2.080602,0.587366,-0.382092,1,False,False,False,...,True,False,True,False,False,False,False,False,True,False


In [ ]:
#Split the dataset into the training set and test set, which lets us evaluate the performance of the algorithms.
X = df.drop("Condition",axis=1) #independent variables
y = df['Condition'] #dependent/target variable


# Split into training (70%) and temporary (30%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) #a training dataset is exactly what it sounds like

# Split temporary set equally into validation and testing (15% each of total data)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) #a validation dataset is supposed to be like a secondary training phase, where we can again change behaviour.
#testing dataset is as it sounds

print("Training set size:", X_train.shape[0])
print("Validation set size:", X_val.shape[0])
print("Testing set size:", X_test.shape[0])


Training set size: 211
Validation set size: 45
Testing set size: 46


In [ ]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()
log.fit(X_train, y_train)
y_pred1 = log.predict(X_test) #using test data here and from now on, should probably switch to val and then rerun or something?
accuracy_score(y_test,y_pred1)

0.8260869565217391

In [ ]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
y_pred2 = svm.predict(X_test)
accuracy_score(y_test,y_pred2)
#

0.8260869565217391

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred3 = knn.predict(X_test)
accuracy_score(y_test,y_pred3)


0.8695652173913043

In [111]:
#nonlinear ML algos, dont need preprocessing, encoding (turning chest pain into a vector) or feature scaling
data = pd.read_csv('heart.csv')
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [112]:
data.drop_duplicates()
data.shape

(303, 14)

In [114]:
X = data.drop('target', axis=1)
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [115]:
#DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_pred4 = dt.predict(X_test)
accuracy_score(y_test,y_pred4)

0.7362637362637363

In [116]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred5 = rf.predict(X_test)
accuracy_score(y_test,y_pred5)

0.7912087912087912